# Intermediate Topics:

1. Understanding variable assignments: references vs copies
    * Mutable vs immutable objects
    * `copy.copy` and `copy.deepcopy`
2. Function keyword arguments
3. List comprehensions

# 1. Mutable and Immutable data

When we assign a value to a variable in Python a couple of different things could be happening depending on what the value of the variable is and what kind of data type it is, a mutable or immutable datatype (alterable data or inalterable data).

Mutable types in Python:
* `list`
* `dict` (coming in Lesson 5)
* `set`

Immutable types in Python:
* `str`
* `int`
* `float`
* `tuple`

Notice how all the mutable types are "collection" types and (most) of the immutable types are "atomic" (`tuple` is a collection type)?

### What does it mean for `str` to be "immutable"? 

I can do this, yeah?:

```python
my_string = "Excelsior!"
my_string = my_string.replace("E", "X") # "Xxcelsior!"
print(my_string)
```

This means the `str` in `my_string` was altered, right? Not quite. What happened was that `my_string` was created (`Excelsior!`) and then a _new_ string was created with the value of `"Xxcelsior!"`. The value of `my_string` was then _over-written_ with the new string. `Excelsior!` itself was not changed and it remained `Excelsior!` but because no variable had that value any more, it was deleted from memory by the garbage collector.

Similar with `float` and `int`: when you do this...
```python
a = 4
b = 3.4
c = a + b # 7.4
c = c + 1 # 8.4
```

The actual value of `7.4` was not changed. It remained `7.4` but a _new_ `float` with the value of `8.4` was created and then it was assigned as the value of `c`. The value of `c` was overwritten with the new value.

### What does it mean for `list` to be "mutable"?

If I do this:

```python
my_list = ["cat", "hat"]
my_list.append("bat")
print(my_list) # ["cat", "hat", "bat"]
```

I see that `my_list` now has the appended value. But, how do I know that `my_list`, itself, changed and I didn't just get a new `list` with the new value appended?

Because of this:

```python
my_list = ["cat", "hat"]
my_list.append("bat") # <------- It's not: my_list = my_list.append("bat")
```

vs.

```python
my_string = "Excelsior!"
my_string = my_string.replace("E", "X") # <----- It is: my_string = my_string.replace("E", "X")
```

When we append something to a `list`, we are changing the list _in-place_. Mutable types can be changed _in-place_ whereas immutable types create a new, changed object when "altered".

## Variable assignment does not create a copy

In Python, variable assignment merely _binds_ the variable name to the value. It does not make a duplicate or copy.

In [32]:
a = "cat"
b = "cat"
print(a == b) # Testing to see if the _value_ of `a` is the same as the _value_ of `b`
print(a is b) # Testing to see if `a` is pointing to the same object in memory as `b`

True
True


Say we create `'cat'` in variable `a`. 

To create variable `b`, we start with `'cat'` and append `'bat'` to it (now, `'catbat'`). 

Then we call `.replace('bat', '')` so we end up with just `'cat'`. Are both variables now pointing to the same `'cat'`?

In [33]:
a = "cat"
b = ("cat" + "bat").replace("bat", "")
print(a == b)
print(a is b) # False, these are different "cat"

True
False


Now, the `'cat'` in `a` and the `'cat'` in `b` are different objects in memory. This is possible because `str` is _immutable_ and new objects are created in memory as a result of transformations.

## Lists as mutable objects: Two variables pointing to the _same_ data

`a` and `b` are two lists with the same content but created separately

In [34]:
a = ["cat", "bat"]
b = ["cat", "bat"]
print(a == b) # They have the same value
print(a is b) # But they point to different data in memory

True
False


If we append `'hat'` to the list `a`, we do not expect `b` to change because `b` is a separate object in memory.

In [35]:
a.append("hat")

print(a == b) # No longer the same value
print(a is b) # Certainly not the same object in memory

print(a) # The new value of a
print(b) # The original value of b

False
False
['cat', 'bat', 'hat']
['cat', 'bat']


**Now, note the following:**

`a` is a list and `b` is assigned the same value. 

`b` is _referencing_ `a` and they are both pointing to the same object in memory.

Writing `b = a` does NOT create a copy of `a`!

In [36]:
a = ["cat", "bat"]
b = a
print(a == b) # Their values are equal
print(a is b) # And! They are pointing to the same object in memory

True
True


If we append `hat` to the list`a`, we do expect `b` to change, also.

In [37]:
a.append("hat")
print(a)
print(a == b) # b magically has the same value as a!
print(a is b) # In fact, b still is pointing to the same variable as a!
print(b)

['cat', 'bat', 'hat']
True
True
['cat', 'bat', 'hat']


**This is possible only because `list` is a mutable data type. This same behaviour can occur with all mutable data types because they can change _in-place_. It will not occur with immutable data types because new objects are created when an immutable object is "altered".**

## Ok, Python, that's weird. I want to make _copies_ of my data. How?

You can use the `copy` module in the Python standard library:

```python
import copy
a = ["cat", "bat"]
b = copy.copy(a)
print(a == b)
print(a is b)
```

In [38]:
import copy
a = ["cat", "bat"]
b = copy.copy(a)
print(a == b)
print(a is b)

True
False


### Okay, cool. But watch this!

In [39]:
import copy
a = ["cat", ["bat"]] # Second item in the list is a list with one str in it.
b = copy.copy(a)

a[1].append("hat") # Append onto the second item in `a` list
print(b)

['cat', ['bat', 'hat']]


**What?! I thought we copied the list?! Why is `b` still changing when I change `a`??**

Using `copy.copy()` only creates a _shallow_ copy. That is, it copies only the _outer_ list but not creating copies of all the data inside the list. To do that, you need `copy.deepcopy()`

In [40]:
import copy
a = ["cat", ["bat"]]
b = copy.deepcopy(a)
a[1].append("hat")

print(a)
print(b) # b is different now

['cat', ['bat', 'hat']]
['cat', ['bat']]


## To recap (tl;dr):

1. Python does not create copies of data through variable assignment; variable assignment only binds variable names to values
2. Alterations or transformations to immutable data creates new immutable values (i.e. the transformation returns a new value)
3. Alterations or transformations to mutable data are performed in-place (i.e. the transformation only returns None)
4. To create an actual copy of mutable data, you must use `copy.deepcopy()`

# 2. Function keyword arguments

As we have been writing functions so far, we have been creating functions with _positional_ arguments: the function knows which-arguments-are-which purely based on the order they are entered.

e.g.

```python
def my_func(a: str, b: int, c: bool) -> str:
    """
    Returns a sentence describing the values of the
    arguments 'a', 'b', and 'c'.
    """
    return f"The string provided is '{a}'; the integer provided was '{b}'; the boolean provided was '{c}'"
```

In [1]:
def my_func(a: str, b: int, c: bool) -> str:
    """
    Returns a sentence describing the values of the
    arguments 'a', 'b', and 'c'.
    """
    return f"The string provided is '{a}'; the integer provided was '{b}'; the boolean provided was '{c}'"

If we try to run `my_func` without all of the variables, then we will get an error

In [42]:
my_func("Cat!", 43)

TypeError: my_func() missing 1 required positional argument: 'c'

We have to give all arguments:

In [43]:
my_func("Cat!", 43, False)

"The string provided is 'Cat!'; the integer provided was '43'; the boolean provided was 'False'"

I can, conceivably, enter the arguments out of order and the function will still run. Remember, while we used type annotations to describe the type of data we were expecting for each argument, Python does NOT enforce them.

But, the output of my function will be wrong because I put the inputs in the wrong order.

In [44]:
my_func(43, True, "Cat!")

"The string provided is '43'; the integer provided was 'True'; the boolean provided was 'Cat!'"

### What if you wanted an argument to be optional? 

Say, you wanted the function to do something extra only if an extra argument was provided.

This can be done through _keyword arguments_.

## Keyword Arguments

Using the function below, I can run this function even without providing any arguments

In [50]:
def my_func(a: str = "My default string", b: int = 10, c: bool = True):
    """
    Returns a sentence describing the values of the
    arguments 'a', 'b', and 'c'.
    """
    return f"The string provided is '{a}'; the integer provided was '{b}'; the boolean provided was '{c}'"

In [51]:
my_func()

"The string provided is 'My default string'; the integer provided was '10'; the boolean provided was 'True'"

Or, I can change just some of the arguments:

In [52]:
my_func(b=43, c=False)

"The string provided is 'My default string'; the integer provided was '43'; the boolean provided was 'False'"

Or, I can change them all and enter them out of order

In [53]:
my_func(b=23, c=False, a="A new string")

"The string provided is 'A new string'; the integer provided was '23'; the boolean provided was 'False'"

But notice how we still get a correct output even though our arguments are entered in the "wrong" order?

**Note: The way Python works with keyword arguments is that they have to appear _after_ any positional arguments in the function signature.**

This will cause an error because I am using keyword arguments _first_ before the positional arguments. They must appear _last_.

In [54]:
def my_func(a: str = "My default string", b: int, c: bool): # b and c are positional arguments
    """
    Returns a sentence describing the values of the
    arguments 'a', 'b', and 'c'.
    """
    return f"The string provided is '{a}'; the integer provided was '{b}'; the boolean provided was '{c}'"

SyntaxError: non-default argument follows default argument (<ipython-input-54-885cd3bc57f7>, line 1)

This is ok:

In [55]:
def my_func(a: str, b: int = 10, c: bool  = True): # a is the positional argument
    """
    Returns a sentence describing the values of the
    arguments 'a', 'b', and 'c'.
    """
    return f"The string provided is '{a}'; the integer provided was '{b}'; the boolean provided was '{c}'"

Now, when calling the function, the first argument _has_ to be entered but the second and third arguments are optional

In [56]:
my_func("This arg is not optional", c=False)

"The string provided is 'This arg is not optional'; the integer provided was '10'; the boolean provided was 'False'"

**Note: If I am only entering the third argument `c`, then I have to be clear about which argument I am entering by stating `c=False`. If I just entered `False`, Python will assume I am entering the `b` argument because it comes after `a`. Even if the arguments are keyword args, they also still have an order.**

## To recap (tl;dr):

1. When writing your function signature, you can indicate that some arguments are optional by providing a default value: `def my_func(a: str = "my default value", ...) -> ...:`
2. Keyword arguments can be entered out of order so long as they declared specifically: `my_func(c=False, a="My new str")`
3. When using keyword arguments in your function signature, you must declare your positional (non-optional) arguments _first_, then your keyword (optional) arguments

# 3. List Comprehensions

List comprehensions are a powerful feature of Python. It provides a way of doing simple loop, transformation, and/or filtering in one line instead of making a full loop.

An example:

In [4]:
fruits = ["apple", "pear", "guava", "boysenberry"]
loud_fruits = [] # my accumulator
for fruit in fruits:
    if "a" in fruit:
        loud_fruits.append(fruit.upper() + "!")
        
print(loud_fruits)

['APPLE!', 'PEAR!', 'GUAVA!']


Using a list comprehension, this can become:

In [5]:
loud_fruits = [fruit.upper() + "!" for fruit in fruits if "a" in fruit]
print(loud_fruits)

['APPLE!', 'PEAR!', 'GUAVA!']


### List Comprehension syntax

It took me a while to get this firmly stuck in my brain. So, don't feel bad if you find yourself Google-ing "Python list comprehension syntax" from time to time.

```python
[<do this thing> for <item> in <collection>]

# Optionally, you can also have a basic if for filtering

[<do this thing with item> for <item> in <collection> if <condition check with item>]
```

**Some Examples**

Sometimes the "do this thing" part is just that you want the item added to the list if some condition is met. It would look like this:

```python
[item for item in collection if "a" in item] # For example
```

Other times, you may want to do a simple transformation, like this:

```python
[item.title() for item in collection]
```

You can use list comprehensions to filter out empty strings from a list effectively:

```python
[string for string in my_list_of_strings if string]
```

### Benefits of list comprehensions

1. Less typing
2. Simple ideas can be expressed and executed on one line (can be easier to read than a full loop)
3. Slight performance gain because the Python parser reads the whole loop on one line and can optimize how it compiles into bytecode because it has all the conditions of the loop at once.

### How far can you go with a list comprehension? Can I do "too much"? 

**Yes, you can do too much**

For example:

```python
matrix_data = [[1, 2, 3],
               [4, 5, 6], 
               [7, 8, 9]]

double_evens_with_odds_negative = [2*j if j%2 == 0 else -j for i in matrix_data for j in i]
```

This kind of logic can get hard to follow when reading it all on one line. It might be easier to follow in a full `for` loop:

```python
double_evens_with_odds_negative = []
for i in matrix_data:
    for j in i:
        if j%2 == 0:
            double_evens_with_odds_negative.append(2*j)
        else:
            double_evens_with_odds_negative.append(-j)
```

But, here is the syntax do such magery:

```python
[statement for sublist in mainlist for item in sublist]
```

In this case we used the following statement in place of `<do this with item`:

```python
2*j if j%2 == 0 else -j
```

The double for loop is accomplished in the list comprehension in the same order you would use if you were to write it out in full:

e.g. 

```python
for sublist in mainlist:
    for item in sublist:
        <do this with item>
```

Becomes:

```python
[<do this with item> for sublist in mainlist for item in sublist]
```

## To recap (tl;dr):

1. List comprehensions are a Python feature that can make your code more expressive and faster to write
2. They can be used for quick transformations and/or filtering of data in a collection
3. The basic syntax: `[<do this with item> for item in collection <optional condition with item>]`
4. You _can_ do too much with a list comprehension. As a general rule, if it does not fit on one line, break it up into a regular `for` loop.

# One final note about keyword arguments and list comprehensions in your workbooks

Because this first iteration of the Python course has students of multiple abilities, please refrain from using keyword arguments and list comprehensions in the workbooks you submit until _after_ **Workbook 07** until I have had an opportunity to introduce them to everyone. Feel free to play around with them in your workbooks but try to refrain from using them in the workbook that you will be having reviewed by someone else who may be utterly confused if they see them without having been introduced to them.

Thanks for your interest in Python and this course so far! Hopefully, you found this additional information useful and satisfying. 🐍
